In [3]:
from datasets import load_dataset

In [4]:
dataset = "ag_news"
ag = load_dataset(dataset)

In [5]:
ag['train'][0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2}

In [10]:
## print dataset "text" max length

text = ag['train']['text']
## print max length
max_len = max([len(t) for t in text])
print("Max length:", max_len)

Max length: 1012


In [2]:
from transformers import AutoModelForCausalLM

model_name = "Qwen/Qwen1.5-MoE-A2.7B"
model = AutoModelForCausalLM.from_pretrained(model_name)
print(model)

/home/feihm/llm-fei/.llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipping import of cpp extensions due to incompatible torch version 2.9.0+cu128 for torchao version 0.13.0
Loading checkpoint shards: 100%|██████████| 8/8 [00:08<00:00,  1.11s/it]


Qwen2MoeForCausalLM(
  (model): Qwen2MoeModel(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-23): 24 x Qwen2MoeDecoderLayer(
        (self_attn): Qwen2MoeSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): Qwen2MoeRotaryEmbedding()
        )
        (mlp): Qwen2MoeSparseMoeBlock(
          (gate): Linear(in_features=2048, out_features=60, bias=False)
          (experts): ModuleList(
            (0-59): 60 x Qwen2MoeMLP(
              (gate_proj): Linear(in_features=2048, out_features=1408, bias=False)
              (up_proj): Linear(in_features=2048, out_features=1408, bias=False)
              (down_proj): Linear(in_features=1408, out_features=2048, bias=False)
        

In [12]:
PROMPT_TEMPLATE.format(text=ag['test']['text'][0])

"You are a helpful news classifier. Classify the following news article into one of four categories: World, Sports, Business, Sci/Tech.\n\nNews: Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.\n\nLabel:"

In [16]:
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
LABEL_MAP = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}
LABEL_LIST = [LABEL_MAP[i] for i in range(4)]

PROMPT_TEMPLATE = (
    "You are a helpful news classifier. "
    "Classify the following news article into one of four categories: World, Sports, Business, Sci/Tech.\n\n"
    "News: {text}\n\nLabel:"
)
samples_text = ag['test']['text'][:10]
samples_label = ag['test']['label'][:10]

correct = 0
for i, s in enumerate(range(10)):
    print(s)
    prompt = PROMPT_TEMPLATE.format(text=samples_text[i])
    inputs = tokenizer(prompt, return_tensors='pt')
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=32, do_sample=False, eos_token_id=tokenizer.eos_token_id)
    gen_ids = out[0][inputs['input_ids'].shape[-1]:]
    gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

    pred = None
    gt = LABEL_MAP[samples_label[i]]
    gen_low = gen_text.lower()
    for lab in LABEL_LIST:
        if lab.lower() in gen_low:
            pred = lab
            break
    if pred is None:
        first_word = gen_text.splitlines()[0].split()[0] if gen_text.strip() else ""
        for lab in LABEL_LIST:
            if lab.lower().startswith(first_word.lower()):
                pred = lab
                break
    if pred is None:
        pred = "Unknown"

    correct += int(pred == gt)
    print(f"{i}: true={gt}, pred={pred}, gen={gen_text[:200]}")

print(f"Accuracy on {len(samples_text)} samples: {correct}/{len(samples_text)} = {correct/len(samples_text):.2f}")



0
0: true=Business, pred=Business, gen=Business

The news article is about the concerns for the pension of workers at Turner Newall, a company that is facing financial difficulties. The article mentions that unions
1
1: true=Sci/Tech, pred=Business, gen=Business

Explanation: The news article is about a competition for a prize related to private space flight. The article mentions a second team of rocketeers competing for
2
2: true=Sci/Tech, pred=Business, gen=Business

Explanation: The news article is about a company that has won a grant to study peptides, which are short chains of amino acids. The company was founded
3
3: true=Sci/Tech, pred=Business, gen=Business

Explanation: The news article is about a prediction unit that helps forecast wildfires. The article discusses the work of Mike Fitzpatrick, who uses maps, figures
4
4: true=Sci/Tech, pred=Business, gen=Business

Explanation: The news article is about a new rule adopted by Southern California's smog-fighting agency to reduce

In [1]:
from transformers import AutoModel
from deepspeed.runtime.zero.stage3 import estimate_zero3_model_states_mem_needs_all_live


/home/feihm/llm-fei/.llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-10-23 12:44:46,670] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/feihm/llm-fei/.llm/lib/python3.10/site-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [2]:
model = AutoModel.from_pretrained('Qwen/Qwen1.5-MoE-A2.7B')
estimate_zero3_model_states_mem_needs_all_live(model, num_gpus_per_node=4, num_nodes=1)

Loading checkpoint shards: 100%|██████████| 8/8 [00:09<00:00,  1.13s/it]

Estimated memory needed for params, optim states and gradients for a:
HW: Setup with 1 node, 4 GPUs per node.
SW: Model with 14004M total params, 311M largest layer params.
  per CPU  |  per GPU |   Options
  352.16GB |   1.16GB | offload_param=cpu , offload_optimizer=cpu , zero_init=1
  352.16GB |   1.16GB | offload_param=cpu , offload_optimizer=cpu , zero_init=0
  313.03GB |   7.68GB | offload_param=none, offload_optimizer=cpu , zero_init=1
  313.03GB |   7.68GB | offload_param=none, offload_optimizer=cpu , zero_init=0
    6.96GB |  59.85GB | offload_param=none, offload_optimizer=none, zero_init=1
  313.03GB |  59.85GB | offload_param=none, offload_optimizer=none, zero_init=0
